In [1]:
import os
from dotenv import load_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationStringBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

In [2]:
import omegaconf

# Load the config file
config = omegaconf.OmegaConf.load("../config/main.yaml")

In [3]:
instructions = config['prompt_text_to_melody']

In [4]:
intro[0:100]

NameError: name 'intro' is not defined

In [5]:
load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

In [21]:
temperature=0.9
llm_model = "gpt-4-32k-0613"

In [22]:
llm = ChatOpenAI(temperature=temperature, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [23]:
conversation.predict(input=instructions)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I need assistance in producing AI-generated text that I convert to music using MIDI files. Initially, I'll provide a description of the format I need for the textual representation of the music.  Since music is a time-based art form, the notes follow each other in time, and sometimes there are no notes, that is, silences.  The way I would like you to generate them is as follows: Each note is represented as a tuple of two elements: The pitch of the note (integer value).  Because I will use this text representation and convert to MIDI the note should be a number from 21 (that is note A0 - 27,50 Hz) to 96 (that is C7 - 2093 hz) so use these numbers to represent th

InvalidRequestError: The model `gpt-4-32k-0613` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

In [24]:
composition = conversation.predict(input="Write a basic melody inspired in Beatles music")
composition



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Write a basic melody inspired in Beatles music
AI:


InvalidRequestError: The model `gpt-4-32k-0613` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.

### Example of output parser using LangChain

Let's create an LLM output JSON and use LangChain to parse that output.
Let's extract the melody data and format that output into a JSON format. 

In [25]:
review_template = """\
For the following text, extract the following information:

description: Extract any sentences describing the melody \
an output them as a text.

melody_pitch_duration_data: Extract the composed melody, \
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
description
melody_pitch_duration_data

text: {text}
"""

In [10]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ndescription: Extract any sentences describing the melody an output them as a text.\n\nmelody_pitch_duration_data: Extract the composed melody, and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ndescription\nmelody_pitch_duration_data\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [11]:
messages = prompt_template.format_messages(text=composition)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

NameError: name 'composition' is not defined

### Parse the LLM output string into a Python dictionary

In [12]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [13]:
description_schema = ResponseSchema(name="description",
                             description="Extract any sentences describing the melody \
                            an output them as a text.")
melody_pitch_duration_data_schema = ResponseSchema(name="melody_pitch_duration_data",
                                    description="Extract the composed melody, \
                                    and output them as a comma separated Python list.")

response_schemas = [description_schema, 
                    melody_pitch_duration_data_schema]

In [14]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [15]:
format_instructions = output_parser.get_format_instructions()

In [16]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // Extract any sentences describing the melody                             an output them as a text.
	"melody_pitch_duration_data": string  // Extract the composed melody,                                     and output them as a comma separated Python list.
}
```


In [17]:
review_template_2 = """\
For the following text, extract the following information:

description: Extract any sentences describing the melody \
an output them as a text.

melody_pitch_duration_data: Extract the composed melody, \
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
description
melody_pitch_duration_data

text: {text}

{format_instructions}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=composition, 
                                format_instructions=format_instructions)

NameError: name 'composition' is not defined

In [19]:
print(messages[0].content)

NameError: name 'messages' is not defined

In [20]:
response = chat(messages)

NameError: name 'chat' is not defined

In [179]:
print(response.content)

```json
{
	"description": "Absolutely, let's generate another melody inspired by The Beatles, this time I'll vary the motif more greatly for more interesting movement. I'll be using elements persistent in their music such as a clear tonal center, repetitive elements, and distinct interactions with the bass line. This melody will also be in the key of G major, another commonly used key in The Beatles' music. This melody introduces more leap intervals, which adds interest and energy to the melody. The use of syncopation and distinctive rythmic patterns is also characteristic of many Beatles' melodies. The repeating motifs and variations are designed to give the melody a catchy, memorable quality.",
	"melody_pitch_duration_data": "[ (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.75), (62, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (0, 0.75), (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.5), (67, 0.75), (64, 0.25),

In [180]:
output_dict = output_parser.parse(response.content)

In [181]:
output_dict

{'description': "Absolutely, let's generate another melody inspired by The Beatles, this time I'll vary the motif more greatly for more interesting movement. I'll be using elements persistent in their music such as a clear tonal center, repetitive elements, and distinct interactions with the bass line. This melody will also be in the key of G major, another commonly used key in The Beatles' music. This melody introduces more leap intervals, which adds interest and energy to the melody. The use of syncopation and distinctive rythmic patterns is also characteristic of many Beatles' melodies. The repeating motifs and variations are designed to give the melody a catchy, memorable quality.",
 'melody_pitch_duration_data': '[ (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.75), (62, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (0, 0.75), (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.5), (67, 0.75), (64, 0.25), (62, 0.5)

In [183]:
melody_data = output_dict.get('melody_pitch_duration_data')
melody_data

'[ (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.75), (62, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (0, 0.75), (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.5), (67, 0.75), (64, 0.25), (62, 0.5), (59, 0.5), (57, 0.5), (0, 2) ]'

In [188]:
import ast

result = ast.literal_eval(melody_data)
print(result)

[(55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.75), (62, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (0, 0.75), (55, 0.25), (57, 0.25), (59, 0.5), (57, 0.5), (55, 0.5), (59, 0.5), (62, 0.5), (64, 0.5), (67, 0.75), (64, 0.25), (62, 0.5), (59, 0.5), (57, 0.5), (0, 2)]
